In [1]:
# !pip install pandas 
# !pip install tdqm
# !pip install pandarallel

In [2]:
import re
import string
import math
import json
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

In [3]:
_WORD_SPLIT = re.compile("([.,!?\"/':;)(])")
_DIGIT_RE = re.compile(br"\d")
STOP_WORDS = "\" \' [ ] . , ! : ; ?".split(" ")

def basic_tokenizer(sentence):
    words = []
    for space_separated_fragment in sentence.strip().split():
        words.extend(_WORD_SPLIT.split(space_separated_fragment))
    return [w.lower() for w in words if w != '' and w != ' ' and w not in string.punctuation]

def remove_appending_title(text,title):
    return text.replace(f"{title}\n\n{title}",f"{title} ")

def create_sliding_window(text, size=256, overlap=32):
    actual_size = size - overlap
    windows = []
    n_windows = math.ceil(len(text)/actual_size)
    for i in range(n_windows):
        windows.append(" ".join(text[i*actual_size:i*actual_size + size]))
    return windows

In [4]:
all_titles = []
all_texts = []
all_bm25_texts = []
with open("../data/raw/wikipedia_20220620_cleaned/wikipedia_20220620_cleaned.jsonl", encoding="utf-8") as f:
    for i,line in tqdm(enumerate(f)):
        x = json.loads(line)
        text = remove_appending_title(x["text"],x["title"])
        text = text.split(" ")
        sliding_windows = create_sliding_window(text)
        all_texts.extend(sliding_windows)
        # all_bm25_texts.extend(bm25_windows)
        all_titles.extend([x['title'],]*len(sliding_windows))

0it [00:00, ?it/s]

In [5]:
df = pd.DataFrame()
df["title"] = all_titles
df["text"] = all_texts
df

,title,text
0,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s..."
1,Internet Society,Internet Society hay ISOC là một tổ chức quốc...
2,Tiếng Việt,"Tiếng Việt , cũng gọi là tiếng Việt Nam hay Vi..."
3,Tiếng Việt,"hệ thống thanh điệu phát triển cao hơn, hệ thố..."
4,Tiếng Việt,tiếp xúc Hán – Việt thành 2 giai đoạn chính: \...
...,...,...
1944401,VNOI,chuyên viên của các tập đoàn công nghệ lớn như...
1944402,Vòng hoa,"Vòng hoa (; IPA: ) là một loại hoa, lá, quả, ..."
1944403,Vòng hoa,vật trong thần thoại La Mã và Hy Lạp được miêu...
1944404,Vòng hoa,"một vai trò nổi bật, tương tự như ở Anh.\n\n==..."


In [ ]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=6) 

def apply_tokenizer(df, num_chunks):
    chunk_sr = pd.Series()
    i = 0
    end = df.shape[0]
    chunk_size = math.floor(end/num_chunks)
    while i != end:
        if end - i < chunk_size:
            chunk_size = end - i
#         print((i,i+chunk_size))
        temp_df = df[i:i+chunk_size].copy()
        chunk_sr = pd.concat([chunk_sr, temp_df["text"].parallel_apply(lambda x: " ".join(basic_tokenizer(x)))], ignore_index=True)
        i+=chunk_size
    return chunk_sr

df["bm25_text"] = apply_tokenizer(df, 5)

In [7]:
df.head()

,title,text,bm25_text
0,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s...",trang chính <templatestyles src= wiki2021 styl...
1,Internet Society,Internet Society hay ISOC là một tổ chức quốc...,internet society hay isoc là một tổ chức quốc ...
2,Tiếng Việt,"Tiếng Việt , cũng gọi là tiếng Việt Nam hay Vi...",tiếng việt cũng gọi là tiếng việt nam hay việt...
3,Tiếng Việt,"hệ thống thanh điệu phát triển cao hơn, hệ thố...",hệ thống thanh điệu phát triển cao hơn hệ thốn...
4,Tiếng Việt,tiếp xúc Hán – Việt thành 2 giai đoạn chính: \...,tiếp xúc hán – việt thành 2 giai đoạn chính bu...


In [10]:
df.shape

(1944406, 3)

In [9]:
df.to_csv("../data/processed/wikipedia_20220620_cleaned_v2.csv",index=False)